# Realizo la clasificacion usando Gradient Boosting

## Importo librerias necesarias

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
#PCA
from sklearn.decomposition import PCA
#Standard Scaler
from  sklearn.preprocessing  import StandardScaler

## Leo todos los datos

In [3]:
df = pd.read_csv('../../data/final_data/entrenamiento_postulaciones_vistas4.csv', encoding='utf-8', dtype={
    'titulo_analista': np.int8,
    'titulo_tecnico': np.int8,
    'titulo_administrativo': np.int8,    
    'titulo_vendedor': np.int8,          
    'titulo_comercial': np.int8,        
    'titulo_ventas': np.int8,            
    'titulo_sr': np.int8,                
    'titulo_importante': np.int8,        
    'titulo_jefe': np.int8,              
    'titulo_empresa': np.int8,           
    'titulo_asistente': np.int8,         
    'titulo_supervisor': np.int8,         
    'titulo_jr': np.int8,                
    'titulo_contable': np.int8,          
    'desc_experiencia': np.int8,         
    'desc_empresa': np.int8,             
    'desc_trabajo': np.int8,             
    'desc_excluyente': np.int8,          
    'desc_manejo': np.int8,          
    'desc_importante': np.int8,          
    'desc_equipo': np.int8,              
    'desc_tareas': np.int8,              
    'desc_cliente': np.int8,            
    'desc_clientes': np.int8,            
    'desc_conocimientos': np.int8 
})
    
df.head()

,idaviso,esta_online,titulo,descripcion,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,timestamp,valor_educacion,sexo,edad,se_postulo,idpostulante_int
0,1112356526,1,576460752303423488,5850181757109797004,0,0,0,0,0,182,6,qekedek,9.202352e+06,9.0,1,24,0,414724
1,1112346262,1,1154047404513689600,9200867796308722401,0,0,0,0,1,73,480,83zk2R,7.817715e+06,10.0,2,26,1,87038
2,1112397910,0,40532396646334464,5161232066131706144,0,0,0,0,1,152,26,KBaGp2p,6.759920e+06,10.0,1,27,1,180061
3,1112452139,1,9007199254773760,6820714967321280568,0,0,0,0,0,5,17,QNE8J24,4.683675e+06,6.0,2,20,0,236192
4,1112249678,0,576460752303423488,8098035492326473800,1,0,10,0,0,182,2082,JBrpa35,1.290450e+07,10.0,1,26,1,176876


In [4]:
cols = df.head(0)
cols.drop(['se_postulo', 'idaviso', 'idpostulante'],inplace=True, axis=1)
cols = cols.columns

/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
mintimestamp = df['timestamp'].min()
df['timestamp'] = df['timestamp']-mintimestamp
mintimestamp = df['idpostulante_int'].min()
df['idpostulante_int'] = df['idpostulante_int']-mintimestamp

In [6]:
X = np.array(df.loc[:,cols])
y = np.array(df['se_postulo']) 

## Aplico Scaler

In [7]:
scaler = StandardScaler() 
scaler.fit(X)
X = scaler.transform(X) 

## Aplico PCA

In [8]:
pca = PCA(n_components=2)
pca.fit(X)
X = pca.transform(X)

## Realizo el Train Test Split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=7506)

## Defino el clasificador

In [10]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

## Entreno el clasificador

In [11]:
clf.fit(X_train, y_train);
#Set3. Scaler. PCA 1. Gradient Boosting 0.6236741846921211 0.89506
#Set3. Scaler. PCA 2. Gradient Boosting 

## Reviso el score

In [12]:
clf.score(X_test, y_test)

0.6163573642545204

## Predigo con todos los datos

In [13]:
X_full_data = X
pred_full_data = clf.predict(X_full_data)
df['sepostulo'] = pred_full_data

## Armo .csv para subir a Kaggle

In [14]:
top100_full = pd.read_csv("../../test_final_100k.csv", encoding='utf-8')
top100_full = pd.merge(top100_full, df, how='left', on=['idaviso','idpostulante'])
top100_full = top100_full[['id', 'se_postulo', 'sepostulo']]
top100_full.fillna(0, inplace=True)
top100_full['se_postulo'] = top100_full['se_postulo'].astype('int')
top100_full['sepostulo'] = top100_full['sepostulo'].astype('int')
top100_full.loc[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1), 'sepostulo'] = 1
top100_full[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1)]['sepostulo']
top100_full.drop('se_postulo', axis=1, inplace=True)
top100_full.to_csv('../../data/final_data/top100k.csv', encoding='utf-8', index=False)